In [20]:
import findspark
import pandas as pd
from pyspark.sql.functions import col, max, unix_timestamp, from_unixtime, to_date,coalesce, lit
import os
from pyspark.sql import SparkSession

In [21]:
findspark.init()
findspark.find()

'/Users/pamohlyakov/miniconda3/envs/sparke/lib/python3.10/site-packages/pyspark'

In [22]:
spark = SparkSession.builder.master("local[4]").appName("practice").getOrCreate()

23/11/23 20:48:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [23]:
aircraft = spark.read.csv("./air_stats/aircraft.csv", sep=";", inferSchema="true", header="true")
airlines = spark.read.csv("./air_stats/airlines.csv", sep=",", inferSchema="true", header="true")
airports = spark.read.csv("./air_stats/airports.csv", sep=",", inferSchema="true", header="true")
cargo = spark.read.csv("./air_stats/cargo transportation.csv", sep=";", inferSchema="true", header="true")
passenger = spark.read.csv("./air_stats/passenger transportation.csv", sep=";", inferSchema="true", header="true")

In [24]:
aircraft.printSchema()

root
 |-- Вид воздушного судна: string (nullable = true)
 |-- Тип (наименование) воздушного судна: string (nullable = true)
 |-- Опознавательные знаки: string (nullable = true)
 |-- Серийный №: string (nullable = true)
 |-- Идентификац. №: string (nullable = true)
 |-- номер свидетельства о регистрации: string (nullable = true)
 |-- дата действующего свидетельства о регистрации: string (nullable = true)



In [25]:
airlines.printSchema()

root
 |-- Индекс: integer (nullable = true)
 |-- Название: string (nullable = true)
 |-- Юридическое Название: string (nullable = true)
 |-- Самолеты: string (nullable = true)



In [26]:
airports.printSchema()

root
 |-- Наименование аэропорта РФ: string (nullable = true)
 |-- Свидетельство: string (nullable = true)
 |-- Юрлицо: string (nullable = true)
 |-- Тип: string (nullable = true)



In [27]:
cargo.printSchema()

root
 |-- Наименование аэропорта РФ: string (nullable = true)
 |-- Год периода данных: integer (nullable = true)
 |-- Январь: string (nullable = true)
 |-- Февраль: string (nullable = true)
 |-- Март: string (nullable = true)
 |-- Апрель: string (nullable = true)
 |-- Май: string (nullable = true)
 |-- Июнь: string (nullable = true)
 |-- Июль: string (nullable = true)
 |-- Август: string (nullable = true)
 |-- Сентябрь: string (nullable = true)
 |-- Октябрь: string (nullable = true)
 |-- Ноябрь: string (nullable = true)
 |-- Декабрь: string (nullable = true)
 |-- Январь - Декабрь: string (nullable = true)



In [28]:
passenger.printSchema()

root
 |-- Наименование аэропорта РФ: string (nullable = true)
 |-- Год периода данных: integer (nullable = true)
 |-- Январь: string (nullable = true)
 |-- Февраль: string (nullable = true)
 |-- Март: string (nullable = true)
 |-- Апрель: string (nullable = true)
 |-- Май: string (nullable = true)
 |-- Июнь: string (nullable = true)
 |-- Июль: string (nullable = true)
 |-- Август: string (nullable = true)
 |-- Сентябрь: string (nullable = true)
 |-- Октябрь: string (nullable = true)
 |-- Ноябрь: string (nullable = true)
 |-- Декабрь: string (nullable = true)
 |-- Январь - Декабрь: string (nullable = true)



In [29]:
#print(*aircraft.loc[aircraft["Вид воздушного судна"] == "самолет"]["Тип (наименование) воздушного судна"].unique().tolist(),sep='\n')
aircraft.filter(aircraft["Вид воздушного судна"] == "самолет").select("Тип (наименование) воздушного судна").distinct().show()

+-----------------------------------+
|Тип (наименование) воздушного судна|
+-----------------------------------+
|                   Авиатика 890-582|
|                       SportCruiser|
|                              СП-90|
|                               Л-6М|
|                               К-10|
|                          Птенец-2А|
|                              FA-20|
|                        PZL Koliber|
|                                МИР|
|                         Sky Ranger|
|                             Че-257|
|                           Х-32-912|
|                             СП-34К|
|                          Red Liner|
|                         Skycruiser|
|                   KITFOX 5 OUTBACK|
|                             ТУНДРА|
|                      Принцесса 001|
|                             ТР-301|
|                    Птенец 2 РТ 912|
+-----------------------------------+
only showing top 20 rows



In [30]:
aircraft = aircraft.withColumn('дата действующего свидетельства о регистрации', to_date(col('дата действующего свидетельства о регистрации'), 'dd.MM.yyyy'))

In [31]:
max_registration_date = aircraft.agg(max(col("дата действующего свидетельства о регистрации"))).collect()[0][0]
aircraft.filter((col("дата действующего свидетельства о регистрации") == max_registration_date) & 
                           (col("Вид воздушного судна") == "самолет")).show()

+--------------------+-----------------------------------+---------------------+----------+--------------+---------------------------------+---------------------------------------------+
|Вид воздушного судна|Тип (наименование) воздушного судна|Опознавательные знаки|Серийный №|Идентификац. №|номер свидетельства о регистрации|дата действующего свидетельства о регистрации|
+--------------------+-----------------------------------+---------------------+----------+--------------+---------------------------------+---------------------------------------------+
|             самолет|                          E 330 LXK|             RA-2277G|      null|  ЕЭВС.03.2445|                             2126|                                   2017-12-22|
+--------------------+-----------------------------------+---------------------+----------+--------------+---------------------------------+---------------------------------------------+



In [32]:
aircraft.write.parquet("./aircraft.parquet", mode="overwrite")

In [33]:
aircraft = spark.read.parquet("aircraft.parquet")

In [34]:

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession


months = ['Январь', 'Февраль',
       'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь',
       'Октябрь', 'Ноябрь', 'Декабрь']
months_year = months + ['Январь - Декабрь']


spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

def convert_to_float(x):
    if x == "***":
        return None 
    elif isinstance(x, str):
        return float(x.replace(',', '.').replace(' ', ''))
    else:
        return x

convert_to_float_udf = udf(convert_to_float, FloatType())

for m in months_year:
    cargo = cargo.withColumn(m, convert_to_float_udf(cargo[m]))
    passenger = passenger.withColumn(m, convert_to_float_udf(passenger[m]))

23/11/23 20:48:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [35]:
cargo = cargo.withColumnRenamed('Январь - Декабрь', 'Грузопоток за год')
passenger = passenger.withColumnRenamed('Январь - Декабрь', 'Пасажиропоток за год')
cargo = cargo.withColumn('Грузопоток за год', sum(coalesce(col(x), lit(0)) for x in months))
passenger = passenger.withColumn('Пасажиропоток за год', sum(coalesce(col(x), lit(0)) for x in months))
first_merge = airports.select(['Наименование аэропорта РФ', 'Юрлицо']).join(
    passenger.select(['Наименование аэропорта РФ', 'Год периода данных', 'Пасажиропоток за год']), ["Наименование аэропорта РФ"], "inner"
)

company_year_sum = first_merge.join(
    cargo.select(['Наименование аэропорта РФ', 'Год периода данных', 'Грузопоток за год']), ["Наименование аэропорта РФ", "Год периода данных"], "inner"
)
company_year_sum_2018 = company_year_sum.filter(company_year_sum['Год периода данных'] == 2018)
company_year_sum_2018.show()

+-------------------------+------------------+--------------------+--------------------+-----------------+
|Наименование аэропорта РФ|Год периода данных|              Юрлицо|Пасажиропоток за год|Грузопоток за год|
+-------------------------+------------------+--------------------+--------------------+-----------------+
|                     Ухта|              2018|ОАО  «Комиавиатра...|                 0.0|              0.0|
|                   Усинск|              2018|ОАО  «Комиавиатранс»|                 0.0|              0.0|
|                   Печора|              2018|ОАО  «Комиавиатра...|                 0.0|              0.0|
|                   Братск|              2018|    ПАО «АэроБратск»|                 0.0|              0.0|
|                 Усть-Кут|              2018|ОАО «Аэропорт Уст...|                 0.0|              0.0|
|                     Мама|              2018|ООО «Мамский аэро...|                 0.0|              0.0|
|                  Бодайбо|          